# Classification d'opinion avec un Transformer Ollama

Téléchargez les données sur le site d'origine:
[https://ai.stanford.edu/~amaas/data/sentiment/]

Penser à dezipper le fichier d'embeddings présent dans le répertoire data.

Ensuite, le TP est opérationnel :)

In [ ]:
# import standard + 
# 
import numpy as np

import torch
from tqdm.autonotebook import tqdm

import os
import time
import logging
import re
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = "cpu"
print(device)





A. Chargement des données
------------------

Tout le code est fourni. Le cadre est le même que pour la classification de noms: many-to-one. La tâche est de la classification d'opinion (sentiment en anglais)



In [ ]:
from pathlib import Path

DATASET_PATH_TRAIN = Path("data/aclImdb/train")
DATASET_PATH_TEST = Path("data/aclImdb/test")

NB_DOC_MAX = 1000 # par classe
IMDB_CLASSES  = ['neg','pos']
VOC_SIZE = 10000
BATCH_SIZE = 32
MAX_CHAR_SIZE = 1000


labels = dict(zip(IMDB_CLASSES,[0,1]))

def load_data(datapath, classes, max_size=None):
    txts = []
    files = []
    filelabels = []
    for label in classes:
        c = 0
        new = [os.path.join(datapath / label, f) for f in os.listdir(datapath / label) if f.endswith(".txt")]
        files += new
        # filelabels += [labels[label]] * len(new) 
        for file in (datapath / label).glob("*.txt"):
            t = file.read_text()
            txts.append(t if len(t)<MAX_CHAR_SIZE else t[:MAX_CHAR_SIZE])
            filelabels.append(labels[label])
            c+=1
            if max_size !=None and c>=max_size: break

    return txts, files, filelabels
    #     c+=1
    #     if train_max_size !=None and c>train_max_size: break


txts, files, filelabels = load_data(DATASET_PATH_TRAIN, IMDB_CLASSES, max_size = NB_DOC_MAX)
txts_test, files_test, filelabels_test = load_data(DATASET_PATH_TEST, IMDB_CLASSES, max_size = NB_DOC_MAX)

# B. Lien avec Ollama

Charger l'API, un modèle et lancer un prompt

Différents modes d'usage:

In [ ]:
# !pip install ollama

In [ ]:
import ollama

from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='phi3:latest', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

In [ ]:
# Initialisation d'Ollama
ollama.generate(model='phi3:latest', prompt='Why is the sky blue?')



### Tentative de contrainte du format de réponse

Utiliser un prompt qui force le modèle à générer une réponse d'une certaine forme

In [ ]:
def classification_zero_shot(exemple):
    prompt = f"Give the sentiment of the following text: 'positive' or 'negative' :\n\n{exemple}"
    reponse = ollama.generate(model='phi3:latest',prompt=prompt)
    return reponse

print(txts[0])
rep = classification_zero_shot(txts[0])


In [ ]:
print(rep.response)

In [ ]:
ollama.embed(model='phi3:latest', input='The sky is blue because of rayleigh scattering')